In [1]:
import numpy as np
import scipy as sp
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn import metrics, model_selection
from sklearn.model_selection import train_test_split
from sklearn.grid_search import GridSearchCV

/home/artem/.virtualenvs/python3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/artem/.virtualenvs/python3.6/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


<div align="right">
<a href="#placeholder" class="btn btn-default" data-toggle="collapse">Предварительные инструкции (clickable)</a>
</div>
<div id="placeholder" class="collapse">
<ol>
    <li>Во-первых, на разработку baseline-модели не должно уходить много времени (это требование исходит из оценок затрат на проект в целом - большую часть времени все же нужно потратить на основное решение), процесс должен быть простым, на подавляющем большинстве этапов должны использоваться готовые протестированные инструменты. Все это приводит к тому, что baseline-модели - это дешевый способ сделать грубую оценку потенциально возможного качества модели, при построении которого вероятность допущения ошибок относительно невелика.</li>
    <li>Во-вторых, использование моделей разного типа при построении baseline'ов позволяет на раннем этапе сделать предположения о том, какие подходы являются наиболее перспективными и приоритизировать дальнейшие эксперименты.</li>
    <li>Наличие baseline-моделей позволяет оценить, какой прирост качества дают различные преобразования, усложнения, оптимизации и прочие активности, которые вы предпринимаете для построения финального решения.
</li>
    <li>Наконец, если после построение сложного решения оценка его качества будет очень сильно отличаться от оценки качества baseline-моделей, то это будет хорошим поводом поискать в решении ошибки.
</li>
</ol>
</div>

При желании, можно сразу перейти к результату первого сабмита: ([ссылка](#1))

In [2]:
df = pd.read_csv('data/products_sentiment_train.tsv', delimiter='\t', header=None, names=['sentences', 'p/n'])
df_test = pd.read_csv('data/products_sentiment_test.tsv', delimiter='\t', index_col=None)

In [3]:
df.info() # повезло, в данных нет пропусков.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
sentences    2000 non-null object
p/n          2000 non-null int64
dtypes: int64(1), object(1)
memory usage: 31.3+ KB


Для начала попробуем выделить признаки на основе счетчика слов и классифицировать их несколькими способами.
(LogisticRegression, SGDClassifier, LinearSVC)

In [4]:
def do_check(vectorizer, classifier): # just helper function
    pipe = Pipeline(steps = [
        ('vct', vectorizer),
        ('clf', classifier)
    ])
    
    # тут небольшой читинг, параметр фолдов для кросс валидации уже подобранн методом научного тыка.
    score = model_selection.cross_val_score(pipe, df.sentences, df['p/n'], cv=15, scoring='accuracy')
    return score.mean()

In [5]:
print("CV + LogisticRegression: {}".format(do_check(CountVectorizer(), LogisticRegression())))
print("CV + SGDClassifier:      {}".format(do_check(CountVectorizer(), SGDClassifier())))
print("CV + LinearSVC:          {}".format(do_check(CountVectorizer(), LinearSVC())))

CV + LogisticRegression: 0.7729754621742053
CV + SGDClassifier:      0.7455036845845958
CV + LinearSVC:          0.7484698290275666


Картина, которую мы наблюдаем с параметрами по умолчанию мягко говоря бездарная и что с этим делать пока не понятно, но попробуем то же, только на этот раз выделим признаки на основе частотности слов.

In [6]:
print("TV + LogisticRegression: {}".format(do_check(TfidfVectorizer(), LogisticRegression())))
print("TV + SGDClassifier:      {}".format(do_check(TfidfVectorizer(), SGDClassifier())))
print("TV + LinearSVC:          {}".format(do_check(TfidfVectorizer(), LinearSVC())))

TV + LogisticRegression: 0.7740194561482856
TV + SGDClassifier:      0.7699714006862474
TV + LinearSVC:          0.7699642026821917


Результат несколько улучшился, но он по прежнему бездарен. Нужно подбирать параметры. Но мы все еще можем испытать метод ближайших соседей и деревья.

In [7]:
from sklearn.neighbors import KNeighborsClassifier
print("Result with CV: {}".format(do_check(CountVectorizer(), KNeighborsClassifier())))
print("Result with TV: {}".format(do_check(TfidfVectorizer(), KNeighborsClassifier())))

Result with CV: 0.6589881533322225
Result with TV: 0.7249706128732052


In [8]:
from sklearn.tree import DecisionTreeClassifier
print("Result with CV: {}".format(do_check(CountVectorizer(), DecisionTreeClassifier())))
print("Result with TV: {}".format(do_check(TfidfVectorizer(), DecisionTreeClassifier())))

Result with CV: 0.6914841944299916
Result with TV: 0.6664322497786753


Это просто :facepalm:. В общем если мы не хотим сразу переключаться на lstm и подобные решения (вообще есть сомнения по поводу того, что они могут тут сильно улучшить результат, данных очень мало), единственный способ хоть как -то улучшиться на этом этапе, это все же подбор гиперпатамертов (В дальнейшем можно попробовать поработать с данными, но пока интересно понять, что можно получить "из коробки").

Сразу нужно уточнить, что на этапе baseline нам не интересно проводить глубокий анализ и потому мы просто возьмем лучшие результаты "из коробки" и для них уже будем пытаться проводить подбор параметров. Это частотность слов и методы опорных векторов, логистическая регрессия и стохастический градиентный спуск.

In [9]:
tv, clf = TfidfVectorizer(), LogisticRegression()

pipe = Pipeline(steps = [
        ('vct', tv),
        ('clf', clf)
    ])

parameters = {
        'vct__max_df': (0.25, 0.5, 0.75, 1.0),
        'vct__ngram_range': ((1, 3), (1, 2)),
        'clf__C': (0.00001, 0.0001, 0.01, 1),
        'clf__penalty': ('l2', 'l1'),
}

gs = GridSearchCV(pipe, parameters, scoring='accuracy', refit=True)

In [10]:
%%time
gs.fit(df.sentences, df['p/n'])

CPU times: user 39.7 s, sys: 132 ms, total: 39.8 s
Wall time: 39.8 s


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('vct', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
   ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'vct__max_df': (0.25, 0.5, 0.75, 1.0), 'vct__ngram_range': ((1, 3), (1, 2)), 'clf__C': (1e-05, 0.0001, 0.01, 1), 'clf__penalty': ('l2', 'l1')},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [11]:
best_params = gs.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_params[param_name]))

	clf__C: 1
	clf__penalty: 'l2'
	vct__max_df: 0.25
	vct__ngram_range: (1, 2)


In [12]:
print(gs.best_score_)

0.7385


Ну что, отличный результат. Настройка параметров помогла нам получить худший результат, чем на "чистой" моделе. Логистическая регрессия не показывает впечатляющих результатов.

In [13]:
tv, clf = TfidfVectorizer(), SGDClassifier()

pipe = Pipeline(steps = [
        ('vct', tv),
        ('clf', clf)
    ])

parameters = {
        'vct__max_df': (0.25, 0.5, 0.75, 1.0),
        'vct__ngram_range': ((1, 3), (1, 2)),
        'clf__alpha': (0.00001, 0.000001, 0.0000001, 0.00000001),
        'clf__penalty': ('l2', 'elasticnet'),
        'clf__n_iter': (10, 50, 80)
}

gs = GridSearchCV(pipe, parameters, scoring='accuracy', refit=True)

In [14]:
%%time
gs.fit(df.sentences, df['p/n'])

CPU times: user 2min 10s, sys: 56 ms, total: 2min 10s
Wall time: 2min 10s


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('vct', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
   ...   penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'vct__max_df': (0.25, 0.5, 0.75, 1.0), 'vct__ngram_range': ((1, 3), (1, 2)), 'clf__alpha': (1e-05, 1e-06, 1e-07, 1e-08), 'clf__penalty': ('l2', 'elasticnet'), 'clf__n_iter': (10, 50, 80)},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [15]:
best_params = gs.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_params[param_name]))
        
print("Best score: {}".format(gs.best_score_))

	clf__alpha: 1e-05
	clf__n_iter: 80
	clf__penalty: 'l2'
	vct__max_df: 1.0
	vct__ngram_range: (1, 3)
Best score: 0.789


Здесь пусть и не значительно, но все же удалось улучшить результат. Можно попытаться перебирать больше параметров, но я не заметил существенных положительных изменений и не хотелось бы тратить много времени на этом этапе. Так же не вижу особого смысла в добавлении словаря без нормализации и токенизации текста, а просто добавление стоп слов не дает ощутимого эффекта. Поскольку решено пока на этапе baseline уделить больше внимания оценке разнообразных моделей с поверхностным подбором параметров без обработки данных, на этом и остановимся.

In [16]:
tv, clf = TfidfVectorizer(), LinearSVC()

pipe = Pipeline(steps = [
        ('vct', tv),
        ('clf', clf)
    ])

parameters = {
        'vct__max_df': (0.25, 0.5, 0.75, 1.0),
        'vct__ngram_range': ((1, 3), (1, 2)),
        'clf__penalty': ('l2',), #  'l1'
        'clf__loss': ('hinge',) # 'hinge', 'squared_hinge'
}

gs = GridSearchCV(pipe, parameters, scoring='accuracy', refit=True)

In [17]:
%%time
gs.fit(df.sentences, df['p/n'])

CPU times: user 5.46 s, sys: 16 ms, total: 5.47 s
Wall time: 5.47 s


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('vct', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
   ...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'vct__max_df': (0.25, 0.5, 0.75, 1.0), 'vct__ngram_range': ((1, 3), (1, 2)), 'clf__penalty': ('l2',), 'clf__loss': ('hinge',)},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [18]:
# with penalty=l2 and loss=squared_hingle == Best score: 0.7835
best_params = gs.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_params[param_name]))
        
print("Best score: {}".format(gs.best_score_))

	clf__loss: 'hinge'
	clf__penalty: 'l2'
	vct__max_df: 0.75
	vct__ngram_range: (1, 2)
Best score: 0.7845


Видим, что не смотря на то, что "из корбки" стохастический градиент показывал худший результат, чем метод опорных векторов и логистическая регрессия, после базового подбора параметров он показывает лучший результат, а результат логистической регрессии вообще ухудшился.

В итоге будем использовать для предсказаний вычисление частотности слов и стохастический градиент.

Можно дополнительно попытаться нормальзовать матрицу <a id='2'></a>

In [233]:
%%time
tv, clf = TfidfVectorizer(sublinear_tf=True), SGDClassifier(random_state=113)
trf = TfidfTransformer()

pipe = Pipeline(steps = [
        ('vct', tv),
        ('tr', trf),
        ('clf', clf)
    ])
# расчет занимает время, потому просто оставил оптимальные параметры
parameters = {
        'vct__max_df': (0.5,),
        'vct__ngram_range': ((1, 3),),
        'tr__norm': ('l2',),
        'clf__alpha': (1e-05,),
        'clf__penalty': ('l2',),
        'clf__n_iter': (80, )
}

gs = GridSearchCV(pipe, parameters, scoring='accuracy', refit=True, n_jobs=-1, iid=False, cv=10)
gs.fit(df.sentences, df['p/n'])

CPU times: user 536 ms, sys: 88 ms, total: 624 ms
Wall time: 2.37 s


In [234]:
best_params = gs.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_params[param_name]))
        
print("Best score: {}".format(gs.best_score_))

	clf__alpha: 1e-05
	clf__n_iter: 80
	clf__penalty: 'l2'
	tr__norm: 'l2'
	vct__max_df: 0.5
	vct__ngram_range: (1, 3)
Best score: 0.7974934373359334


Результат не значительно, но улучшился. На этом и остановимся.

### подготовка сабмита <a id='1'></a>

In [6]:
predct = gs.predict(df_test.text.values)

In [7]:
result = {}
for indx, elm in enumerate(predct):
    result[indx] = elm

In [8]:
subm = pd.Series(result, name='y')
subm.index.name = 'Id'
subm = subm.reset_index()

In [9]:
subm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
Id    500 non-null int64
y     500 non-null int64
dtypes: int64(2)
memory usage: 7.9 KB


In [10]:
subm.to_csv('submission.csv', sep=',', encoding='utf-8', index=False)

![title](img/subm_01.png)

Результат ужасен и совершенно не интересен, с этим определенно нужно что -то делать!

Но что интересно, если мы перезапустим модель, результат меняется. Не смотря на то, что изменения в 3 знаке не несет никакой ценности, для kaggle это может быть полезно, поскольку дает + 15 мест.

![title](img/subm_02.png)

После небольшого тюнинга параметров имеем финальный результат. Основные изменения параметров, которые потребовались, для упрощения вынесенны в конструкторы классов, за пределы объекта параметров и могут быть изученны [тут](#2)

![final](img/final.png)

В первую очередь на изменение позиции в лидерборде повлияли параметры random_seed у SGDClassifier и подбор параметра cv у GridSearchCV. Так же параметр sublinear_tf=True у TfidfVectorizer, который по сути просто меняет скейлинг на $$ 1 + log(tf) $$ имеет значение и совокупность всех этих изменений, а возможно и просто случайность обеспечили последний результат. (как можно видеть из первых двух скриншотов, изменение позиции существенное, не смотря на то, что модель не изменялась).

### Заключение

Как можно было заметить векторизация на основе частотности слов и стохастический градиент показали лучший результат: 0.7974934373359334. Результат на тесте естественно отличается и довольно существенно. Итоговый результат на kaggle 0.83750, что есть 19 место.

В целом это простой baseline и модель требует серьезной доработки, поскольку результатом остался недоволен. Но на данном этапе этот результат приемлим.

In [9]:
# Возможно получиться сделать лучше.

In [7]:
%%time
tv, clf = TfidfVectorizer(sublinear_tf=True), SGDClassifier(random_state=113)
trf = TfidfTransformer()

pipe = Pipeline(steps = [
        ('vct', TfidfVectorizer(ngram_range=(1, 2))),
        ('tr', trf),
        ('clf', LinearSVC(class_weight='balanced'))
    ])
# расчет занимает время, потому просто оставил оптимальные параметры
parameters = {
        'vct__max_df': (0.5,),
        'vct__ngram_range': ((1, 3),),
        'tr__norm': ('l2',),
#         'clf__alpha': (1e-05,),
#         'clf__penalty': ('l2',),
#         'clf__n_iter': (80, )
}

gs = GridSearchCV(pipe, parameters, scoring='accuracy', refit=True, n_jobs=-1, iid=False, cv=10)
gs.fit(df.sentences, df['p/n'])

CPU times: user 404 ms, sys: 36 ms, total: 440 ms
Wall time: 2.51 s


In [8]:
best_params = gs.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_params[param_name]))
        
print("Best score: {}".format(gs.best_score_))

	tr__norm: 'l2'
	vct__max_df: 0.5
	vct__ngram_range: (1, 3)
Best score: 0.7939583864596614
